In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
with pd.HDFStore('train.h5') as train:
    df = train.get('train')

In [ ]:
print('Shape : {}'.format(df.shape))

In [ ]:
null_counts = df.isnull().sum()

In [ ]:
plt.figure(figsize=(16,8))
plt.xticks(np.arange(len(null_counts))+0.5,null_counts.index,rotation='vertical')
plt.bar(np.arange(len(null_counts)),null_counts)

In [ ]:
feature_columns = df.columns.drop(['id','timestamp','y'])
iterables = [feature_columns,['count','fraction','seq']]
index = pd.MultiIndex.from_product(iterables,names=['feature','stat'])
ids = df.id.unique()
ids.sort()
nan_df = pd.DataFrame(data=None,index=ids,columns=index)

In [ ]:
from itertools import groupby
for name,group in df.groupby('id'):
    for c in feature_columns:
        nan_count = group[c].isnull().sum()
        timespan = len(group[c])
        nan_indices = pd.isnull(group[c]).nonzero()[0]
        nseq = len(list(groupby(enumerate(nan_indices),lambda (i,x):i-x)))
        nan_df.loc[name][c,'count'] = nan_count
        nan_df.loc[name][c,'fraction'] = nan_count * 1.0/timespan
        nan_df.loc[name][c,'seq'] = nseq

In [ ]:
nan_df.head()

In [ ]:
max_nseq = nan_df.xs('seq',level='stat',axis=1).max()
plt.figure(figsize=(16,8))
plt.xticks(np.arange(len(max_nseq))+0.5,max_nseq.index,rotation='vertical')
plt.bar(np.arange(len(max_nseq)),max_nseq)
plt.gca().set_ylim([0,3])
plt.ylabel('maximum number of NaN ranges')

In [ ]:
grouped = df.groupby('id');
for i,(n,g) in enumerate(grouped):
    d = g.isnull().drop(['timestamp','id','y'],axis=1)
    d.index = g.timestamp - g.timestamp.min()
    d.index.name = 'relative timestamp'
    plt.figure(figsize=(16,12))
    plt.title("ID = %d" % n)
    sns.heatmap(d.T,xticklabels=100,cbar=False)
    if i > 10:
        break